In [22]:
import pandas as pd
from pylab import *

FILE = "ACC.txt"

maf = pd.read_table(FILE, skiprows = 4, usecols=["Hugo_Symbol", "Chromosome", "Start_position", "End_position", "Tumor_Sample_Barcode"])
maf.head()

,Hugo_Symbol,Chromosome,Start_position,End_position,Tumor_Sample_Barcode
0,KLHL3,5,136973018,136973018,TCGA-OR-A5J1-01A-11D-A29I-10
1,KLHDC4,16,87742934,87742934,TCGA-OR-A5J1-01A-11D-A29I-10
2,HIST1H3B,6,26031919,26031919,TCGA-OR-A5J1-01A-11D-A29I-10
3,GABRA1,5,161300197,161300197,TCGA-OR-A5J1-01A-11D-A29I-10
4,SCARF1,17,1543262,1543262,TCGA-OR-A5J1-01A-11D-A29I-10


In [4]:
print len(maf)

13129


In [5]:
FILE = "../hw6/hsa.gff3"
df = pd.read_table(FILE, comment="#", usecols=["Chromosome", "Start", "End", "Type", "miR Name"])
df.head()

,Chromosome,Type,Start,End,miR Name
0,chr1,miRNA_primary_transcript,17369,17436,ID=MI0022705;Alias=MI0022705;Name=hsa-mir-6859-1
1,chr1,miRNA,17409,17431,ID=MIMAT0027618;Alias=MIMAT0027618;Name=hsa-mi...
2,chr1,miRNA,17369,17391,ID=MIMAT0027619;Alias=MIMAT0027619;Name=hsa-mi...
3,chr1,miRNA_primary_transcript,30366,30503,ID=MI0006363;Alias=MI0006363;Name=hsa-mir-1302-2
4,chr1,miRNA,30438,30458,ID=MIMAT0005890;Alias=MIMAT0005890;Name=hsa-mi...


In [6]:
chr2df = {}
for chr in [str(i) for i in range(1,23)] + ["X", "Y"]:
    chr2df[chr] = df[df["Chromosome"] == "chr" + chr]
    print "chr%s\thas %d\tmiRNA records" % (chr, len(chr2df[chr]))

chr1	has 398	miRNA records
chr2	has 271	miRNA records
chr3	has 227	miRNA records
chr4	has 144	miRNA records
chr5	has 175	miRNA records
chr6	has 162	miRNA records
chr7	has 198	miRNA records
chr8	has 224	miRNA records
chr9	has 207	miRNA records
chr10	has 167	miRNA records
chr11	has 255	miRNA records
chr12	has 186	miRNA records
chr13	has 97	miRNA records
chr14	has 258	miRNA records
chr15	has 158	miRNA records
chr16	has 194	miRNA records
chr17	has 284	miRNA records
chr18	has 83	miRNA records
chr19	has 376	miRNA records
chr20	has 115	miRNA records
chr21	has 54	miRNA records
chr22	has 118	miRNA records
chrX	has 310	miRNA records
chrY	has 4	miRNA records


In [23]:
def pos2miR(chr, start, end):
    chunk = chr2df[str(chr)]  # shrink the search space to the given chr
    out = []
    for idx, row in chunk.iterrows():
        if row["Start"] <= start <= row["End"] or \
           row["Start"] <=  end  <= row["End"] or \
           (start < row["Start"] and row["End"] < end):
            # get the name (either hsa-mir-### or hsa-miR-###)
            miR = row["Type"] + ";" + str(row["Start"]) + ";" + str(row["End"]) + ";" + row["miR Name"].split("Name=")[1].split(";")[0]
            out.append(miR)
    return out

In [8]:
# test
pos2miR(1, 17370, 17370)

['miRNA_primary_transcript;17369;17436;hsa-mir-6859-1',
 'miRNA;17369;17391;hsa-miR-6859-3p']

In [9]:
# test2
pos2miR("Y", 2477271, 2477370)

['miRNA_primary_transcript;2477232;2477295;hsa-mir-6089-2',
 'miRNA;2477270;2477293;hsa-miR-6089']

In [25]:
c = pd.DataFrame()
ms = pd.DataFrame()
me = pd.DataFrame()
rs = pd.DataFrame()
re = pd.DataFrame()
t = pd.DataFrame()
i = pd.DataFrame()
s = pd.DataFrame()
from collections import Counter
miR2hits = Counter()
for idx, row in maf.iterrows():
    print idx, '\r',
    if row["Chromosome"] == "MT":
        continue
    hits = pos2miR(row["Chromosome"], (row["Start_position"]), (row["End_position"]))
    for hit in hits:
        print row["Hugo_Symbol"], row["Chromosome"], row["Start_position"], row["End_position"], row["Tumor_Sample_Barcode"], "hits on", hit
        c = c.append([row["Chromosome"]])
        ms = ms.append([row["Start_position"]])
        me = me.append([row["End_position"]])
        rs = rs.append([hit.split(";")[1]])
        re = re.append([hit.split(";")[2]])
        t = t.append([hit.split(";")[0]])
        i = i.append([hit.split(";")[3]])
        s = s.append([row["Tumor_Sample_Barcode"]])
        miR2hits[hit] += 1

MIR4732 17 27188561 27188561 TCGA-OR-A5J5-01A-11D-A29I-10 hits on miRNA_primary_transcript;27188551;27188636;hsa-mir-144
MIR34C 11 111384236 111384236 TCGA-OR-A5J8-01A-11D-A29I-10 hits on miRNA_primary_transcript;111384164;111384240;hsa-mir-34c
MIR506 X 146312342 146312342 TCGA-OR-A5JB-01A-11D-A29I-10 hits on miRNA_primary_transcript;146312238;146312361;hsa-mir-506
MIR520C 19 54210730 54210730 TCGA-OR-A5JC-01A-11D-A29I-10 hits on miRNA_primary_transcript;54210707;54210793;hsa-mir-520c
MIR520C 19 54210730 54210730 TCGA-OR-A5JC-01A-11D-A29I-10 hits on miRNA;54210722;54210743;hsa-miR-520c-5p
RP5-963E22.4 20 61809858 61809858 TCGA-OR-A5JW-01A-11D-A29I-10 hits on miRNA_primary_transcript;61809852;61809938;hsa-mir-124-3
MIR192 11 64658676 64658676 TCGA-OR-A5K4-01A-11D-A29I-10 hits on miRNA_primary_transcript;64658609;64658718;hsa-mir-192
MIR192 11 64658676 64658676 TCGA-OR-A5K4-01A-11D-A29I-10 hits on miRNA;64658675;64658695;hsa-miR-192-5p
MIR509-3 X 146341225 146341225 TCGA-OR-A5KB-01A-11D-

In [26]:
pd.Series(miR2hits)

miRNA;146341214;146341235;hsa-miR-509-3-5p                    1
miRNA;54185467;54185488;hsa-miR-520f-3p                       1
miRNA;54210722;54210743;hsa-miR-520c-5p                       1
miRNA;54215536;54215557;hsa-miR-517-5p                        1
miRNA;64658675;64658695;hsa-miR-192-5p                        1
miRNA_primary_transcript;101351039;101351120;hsa-mir-136      1
miRNA_primary_transcript;111384164;111384240;hsa-mir-34c      1
miRNA_primary_transcript;133675371;133675467;hsa-mir-542      1
miRNA_primary_transcript;145078716;145078792;hsa-mir-892b     1
miRNA_primary_transcript;146312238;146312361;hsa-mir-506      2
miRNA_primary_transcript;146341170;146341244;hsa-mir-509-3    1
miRNA_primary_transcript;27188551;27188636;hsa-mir-144        1
miRNA_primary_transcript;54169933;54170016;hsa-mir-512-1      1
miRNA_primary_transcript;54172411;54172508;hsa-mir-512-2      1
miRNA_primary_transcript;54185413;54185499;hsa-mir-520f       1
miRNA_primary_transcript;54210707;542107

In [30]:
TABLE2 = "C:/Users/jtso/Downloads/targetScanS.gz"  # miRNA binding sites
TABLE3 = "C:/Users/jtso/Downloads/wgEncodeGencodePolyaV19 (1).gz"  # polyA sites

In [31]:
hi = pd.read_table(TABLE2)
hi

,#bin,chrom,chromStart,chromEnd,name,score,strand
0,591,chr1,879822,879830,SAMD11:miR-504,90,+
1,591,chr1,900599,900606,KLHL17:miR-299/299-3p,26,+
2,591,chr1,900605,900612,KLHL17:miR-124/506,7,+
3,591,chr1,900933,900941,KLHL17:miR-19,82,+
4,591,chr1,901054,901061,KLHL17:miR-137,14,+
5,592,chr1,990829,990836,AGRN:miR-144,10,+
6,592,chr1,990831,990838,AGRN:miR-27ab,47,+
7,592,chr1,990831,990838,AGRN:miR-128,16,+
8,592,chr1,990834,990841,AGRN:miR-224,50,+
9,593,chr1,1169073,1169080,B3GALT6:miR-340/340-5p,43,+


In [33]:
chrom2hi = {}
for chr in [str(a) for a in range(1,23)] + ["X", "Y"]:
    chrom2hi[chr] = hi[hi["chrom"] == "chr" + chr]
    print "chr%s\thas %d  \t miR genes" % (chr, len(chrom2hi[chr]))

chr1	has 5456  	 miR genes
chr2	has 4157  	 miR genes
chr3	has 3494  	 miR genes
chr4	has 2383  	 miR genes
chr5	has 3058  	 miR genes
chr6	has 2613  	 miR genes
chr7	has 2554  	 miR genes
chr8	has 2073  	 miR genes
chr9	has 2287  	 miR genes
chr10	has 2302  	 miR genes
chr11	has 2784  	 miR genes
chr12	has 3445  	 miR genes
chr13	has 1084  	 miR genes
chr14	has 1884  	 miR genes
chr15	has 1784  	 miR genes
chr16	has 1836  	 miR genes
chr17	has 3095  	 miR genes
chr18	has 1188  	 miR genes
chr19	has 1463  	 miR genes
chr20	has 1550  	 miR genes
chr21	has 538  	 miR genes
chr22	has 1044  	 miR genes
chrX	has 2080  	 miR genes
chrY	has 47  	 miR genes


In [35]:
def pos2miRbind(chr, start, end):
    area = chrom2hi[str(chr)]
    oc = []
    for idx, row in area.iterrows():
        if row["chromStart"] <= start <= row["chromEnd"] or \
           row["chromStart"] <= end <= row["chromEnd"] or \
           (start < row["chromStart"] and row["chromEnd"] < end):
            miR = "miR binding site" + ";" + str(row["chromStart"]) + ";" + str(row["chromEnd"]) + ";" + row["name"]
            oc.append(miR)
    return oc
        
from collections import Counter
miR2hits = Counter()
for idx, row in maf.iterrows():
    print idx, '\r',
    if row["Chromosome"] == "MT":
        continue
    hits = pos2miRbind(row["Chromosome"], row["Start_position"], row["End_position"])
    for hit in hits:
        print row["Hugo_Symbol"], row["Chromosome"], row["Start_position"], row["End_position"], row["Tumor_Sample_Barcode"], "hits on", hit
        c = c.append([row["Chromosome"]])
        ms = ms.append([row["Start_position"]])
        me = me.append([row["End_position"]])
        rs = rs.append([hit.split(";")[1]])
        re = re.append([hit.split(";")[2]])
        t = t.append([hit.split(";")[0]])
        i = i.append([hit.split(";")[3]])
        s = s.append([row["Tumor_Sample_Barcode"]])
        miR2hits[hit] += 1

LPAL2 6 160903734 160903734 TCGA-OR-A5JC-01A-11D-A29I-10 hits on miR binding site;160903734;160903741;LPAL2:miR-376/376ab/376b-3p:2


In [38]:
fh = pd.read_table(TABLE3)
chrom2fh = {}
for chr in [str(a) for a in range(1,23)] + ["X", "Y"]:
    chrom2fh[chr] = fh[fh["chrom"] == "chr" + chr]
    print "chr%s\thas %d  \t miR genes" % (chr, len(chrom2fh[chr]))

def pos2polyA(chr, start, end):
    area = chrom2fh[str(chr)]
    oc = []
    for idx, row in area.iterrows():
        if row["txStart"] <= start <= row["txEnd"] or \
           row["txStart"] <= end <= row["txEnd"] or \
           (start < row["txStart"] and row["txEnd"] < end):
            miR = str(row["name2"]) + ";" + str(row["txStart"]) + ";" + str(row["txEnd"]) + ";" + str(row["name"])
            oc.append(miR)
    return oc
        
from collections import Counter
miR2hits = Counter()
for idx, row in maf.iterrows():
    print idx, '\r',
    if row["Chromosome"] == "MT":
        continue
    hits = pos2polyA(row["Chromosome"], row["Start_position"], row["End_position"])
    for hit in hits:
        print row["Hugo_Symbol"], row["Chromosome"], row["Start_position"], row["End_position"], row["Tumor_Sample_Barcode"], "hits on", hit
        c = c.append([row["Chromosome"]])
        ms = ms.append([row["Start_position"]])
        me = me.append([row["End_position"]])
        rs = rs.append([hit.split(";")[1]])
        re = re.append([hit.split(";")[2]])
        t = t.append([hit.split(";")[0]])
        i = i.append([hit.split(";")[3]])
        s = s.append([row["Tumor_Sample_Barcode"]])
        miR2hits[hit] += 1

chr1	has 8191  	 miR genes
chr2	has 6877  	 miR genes
chr3	has 5158  	 miR genes
chr4	has 3893  	 miR genes
chr5	has 5148  	 miR genes
chr6	has 4712  	 miR genes
chr7	has 3704  	 miR genes
chr8	has 3484  	 miR genes
chr9	has 2937  	 miR genes
chr10	has 3613  	 miR genes
chr11	has 4471  	 miR genes
chr12	has 5465  	 miR genes
chr13	has 2042  	 miR genes
chr14	has 3097  	 miR genes
chr15	has 4023  	 miR genes
chr16	has 4612  	 miR genes
chr17	has 4854  	 miR genes
chr18	has 2054  	 miR genes
chr19	has 4770  	 miR genes
chr20	has 2722  	 miR genes
chr21	has 950  	 miR genes
chr22	has 1784  	 miR genes
chrX	has 3047  	 miR genes
chrY	has 306  	 miR genes
ABCA5 17 67302911 67302912 TCGA-OR-A5JI-01A-11D-A29I-10 hits on polyA_signal;67302907;67302913;535691


In [39]:
c.rename(columns={0:"Chrom"}, inplace =True)
ms.rename(columns={0:"mStart"}, inplace=True)
me.rename(columns={0:"mEnd"}, inplace=True)
rs.rename(columns={0:"rStart"}, inplace=True)
re.rename(columns={0:"rEnd"}, inplace=True)
t.rename(columns={0:"Type"}, inplace=True)
i.rename(columns={0:"ID"}, inplace=True)
s.rename(columns={0:"Tumor Sample Barcode"}, inplace=True)
final = pd.concat([c,ms,me,rs,re,t,i,s], axis=1)
final = final.reset_index(drop=True)
final

,Chrom,mStart,mEnd,rStart,rEnd,Type,ID,Tumor Sample Barcode
0,17,27188561,27188561,27188551,27188636,miRNA_primary_transcript,hsa-mir-144,TCGA-OR-A5J5-01A-11D-A29I-10
1,11,111384236,111384236,111384164,111384240,miRNA_primary_transcript,hsa-mir-34c,TCGA-OR-A5J8-01A-11D-A29I-10
2,X,146312342,146312342,146312238,146312361,miRNA_primary_transcript,hsa-mir-506,TCGA-OR-A5JB-01A-11D-A29I-10
3,19,54210730,54210730,54210707,54210793,miRNA_primary_transcript,hsa-mir-520c,TCGA-OR-A5JC-01A-11D-A29I-10
4,19,54210730,54210730,54210722,54210743,miRNA,hsa-miR-520c-5p,TCGA-OR-A5JC-01A-11D-A29I-10
5,20,61809858,61809858,61809852,61809938,miRNA_primary_transcript,hsa-mir-124-3,TCGA-OR-A5JW-01A-11D-A29I-10
6,11,64658676,64658676,64658609,64658718,miRNA_primary_transcript,hsa-mir-192,TCGA-OR-A5K4-01A-11D-A29I-10
7,11,64658676,64658676,64658675,64658695,miRNA,hsa-miR-192-5p,TCGA-OR-A5K4-01A-11D-A29I-10
8,X,146341225,146341225,146341170,146341244,miRNA_primary_transcript,hsa-mir-509-3,TCGA-OR-A5KB-01A-11D-A30A-10
9,X,146341225,146341225,146341214,146341235,miRNA,hsa-miR-509-3-5p,TCGA-OR-A5KB-01A-11D-A30A-10


In [40]:
final.to_excel("ACChits.xls")

In [41]:
FILE = "C:/Users/jtso/My Documents/LAML1.txt"

maf = pd.read_table(FILE, usecols=["Hugo_Symbol", "Chromosome", "Start_Position", "End_Position", "Tumor_Sample_Barcode"])

FILE = "../hw6/hsa.gff3"
TABLE2 = "C:/Users/jtso/Downloads/targetScanS.gz"  # miRNA binding sites
TABLE3 = "C:/Users/jtso/Downloads/wgEncodeGencodePolyaV19 (1).gz"  # polyA sites

df = pd.read_table(FILE, comment="#", usecols=["Chromosome", "Start", "End", "Type", "miR Name"])

chr2df = {}
for chr in [str(i) for i in range(1,23)] + ["X", "Y"]:
    chr2df[chr] = df[df["Chromosome"] == "chr" + chr]
    
def pos2miR(chr, start, end):
    chunk = chr2df[str(chr)]  # shrink the search space to the given chr
    out = []
    for idx, row in chunk.iterrows():
        if row["Start"] <= start <= row["End"] or \
           row["Start"] <=  end  <= row["End"] or \
           (start < row["Start"] and row["End"] < end):
            # get the name (either hsa-mir-### or hsa-miR-###)
            miR = str(row["Type"]) + ";" + str(row["Start"]) + ";" + str(row["End"]) + ";" + row["miR Name"].split("Name=")[1].split(";")[0]
            out.append(miR)
    return out

c = pd.DataFrame()
ms = pd.DataFrame()
me = pd.DataFrame()
rs = pd.DataFrame()
re = pd.DataFrame()
t = pd.DataFrame()
i = pd.DataFrame()
s = pd.DataFrame()
from collections import Counter
miR2hits = Counter()
for idx, row in maf.iterrows():
    print idx, '\r',
    try:
        hits = pos2miR(row["Chromosome"], (row["Start_Position"]), (row["End_Position"]))
    except:
        print row["Chromosome"]
        continue
    for hit in hits:
        print row["Hugo_Symbol"], row["Chromosome"], row["Start_Position"], row["End_Position"], row["Tumor_Sample_Barcode"], "hits on", hit
        c = c.append([row["Chromosome"]])
        ms = ms.append([row["Start_Position"]])
        me = me.append([row["End_Position"]])
        rs = rs.append([hit.split(";")[1]])
        re = re.append([hit.split(";")[2]])
        t = t.append([hit.split(";")[0]])
        i = i.append([hit.split(";")[3]])
        s = s.append([row["Tumor_Sample_Barcode"]])
        miR2hits[hit] += 1

MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
NT_113923
MT
MT
MT
MT
MT
MT
MT


In [42]:
hi = pd.read_table(TABLE2)

chrom2hi = {}
for chr in [str(a) for a in range(1,23)] + ["X", "Y"]:
    chrom2hi[chr] = hi[hi["chrom"] == "chr" + chr]
    
def pos2miRbind(chr, start, end):
    area = chrom2hi[str(chr)]
    oc = []
    for idx, row in area.iterrows():
        if row["chromStart"] <= start <= row["chromEnd"] or \
           row["chromStart"] <= end <= row["chromEnd"] or \
           (start < row["chromStart"] and row["chromEnd"] < end):
            miR = "miR binding site" + ";" + str(row["chromStart"]) + ";" + str(row["chromEnd"]) + ";" + str(row["name"])
            oc.append(miR)
    return oc
        
from collections import Counter
miR2hits = Counter()
for idx, row in maf.iterrows():
    print idx, '\r',
    try:
        hits = pos2miR(row["Chromosome"], (row["Start_Position"]), (row["End_Position"]))
    except:
        print row["Chromosome"]
        continue
    for hit in hits:
        print row["Hugo_Symbol"], row["Chromosome"], row["Start_Position"], row["End_Position"], row["Tumor_Sample_Barcode"], "hits on", hit
        c = c.append([row["Chromosome"]])
        ms = ms.append([row["Start_Position"]])
        me = me.append([row["End_Position"]])
        rs = rs.append([hit.split(";")[1]])
        re = re.append([hit.split(";")[2]])
        t = t.append([hit.split(";")[0]])
        i = i.append([hit.split(";")[3]])
        s = s.append([row["Tumor_Sample_Barcode"]])
        miR2hits[hit] += 1

MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
NT_113923
MT
MT
MT
MT
MT
MT
MT


In [43]:
fh = pd.read_table(TABLE3)
chrom2fh = {}
for chr in [str(a) for a in range(1,23)] + ["X", "Y"]:
    chrom2fh[chr] = fh[fh["chrom"] == "chr" + chr]
    print "chr%s\thas %d  \t miR genes" % (chr, len(chrom2fh[chr]))

def pos2polyA(chr, start, end):
    area = chrom2fh[str(chr)]
    oc = []
    for idx, row in area.iterrows():
        if row["txStart"] <= start <= row["txEnd"] or \
           row["txStart"] <= end <= row["txEnd"] or \
           (start < row["txStart"] and row["txEnd"] < end):
            miR = str(row["name2"]) + ";" + str(row["txStart"]) + ";" + str(row["txEnd"]) + ";" + str(row["name"])
            oc.append(miR)
    return oc
        
from collections import Counter
miR2hits = Counter()
for idx, row in maf.iterrows():
    print idx, '\r',
    try:
        hits = pos2miR(row["Chromosome"], (row["Start_Position"]), (row["End_Position"]))
    except:
        print row["Chromosome"]
        continue
    for hit in hits:
        print row["Hugo_Symbol"], row["Chromosome"], row["Start_Position"], row["End_Position"], row["Tumor_Sample_Barcode"], "hits on", hit
        c = c.append([row["Chromosome"]])
        ms = ms.append([row["Start_Position"]])
        me = me.append([row["End_Position"]])
        rs = rs.append([hit.split(";")[1]])
        re = re.append([hit.split(";")[2]])
        t = t.append([hit.split(";")[0]])
        i = i.append([hit.split(";")[3]])
        s = s.append([row["Tumor_Sample_Barcode"]])
        miR2hits[hit] += 1

chr1	has 8191  	 miR genes
chr2	has 6877  	 miR genes
chr3	has 5158  	 miR genes
chr4	has 3893  	 miR genes
chr5	has 5148  	 miR genes
chr6	has 4712  	 miR genes
chr7	has 3704  	 miR genes
chr8	has 3484  	 miR genes
chr9	has 2937  	 miR genes
chr10	has 3613  	 miR genes
chr11	has 4471  	 miR genes
chr12	has 5465  	 miR genes
chr13	has 2042  	 miR genes
chr14	has 3097  	 miR genes
chr15	has 4023  	 miR genes
chr16	has 4612  	 miR genes
chr17	has 4854  	 miR genes
chr18	has 2054  	 miR genes
chr19	has 4770  	 miR genes
chr20	has 2722  	 miR genes
chr21	has 950  	 miR genes
chr22	has 1784  	 miR genes
chrX	has 3047  	 miR genes
chrY	has 306  	 miR genes
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
NT_113923
MT
MT
MT
MT
MT
MT
MT


In [44]:
c.rename(columns={0:"Chrom"}, inplace =True)
ms.rename(columns={0:"mStart"}, inplace=True)
me.rename(columns={0:"mEnd"}, inplace=True)
rs.rename(columns={0:"rStart"}, inplace=True)
re.rename(columns={0:"rEnd"}, inplace=True)
t.rename(columns={0:"Type"}, inplace=True)
i.rename(columns={0:"ID"}, inplace=True)
s.rename(columns={0:"Tumor Sample Barcode"}, inplace=True)
final = pd.concat([c,ms,me,rs,re,t,i,s], axis=1)
final = final.reset_index(drop=True)
final

""


In [45]:
final.to_excel("LAMLhits.xls")

In [46]:
final

""


In [47]:
hi.head()

,#bin,chrom,chromStart,chromEnd,name,score,strand
0,591,chr1,879822,879830,SAMD11:miR-504,90,+
1,591,chr1,900599,900606,KLHL17:miR-299/299-3p,26,+
2,591,chr1,900605,900612,KLHL17:miR-124/506,7,+
3,591,chr1,900933,900941,KLHL17:miR-19,82,+
4,591,chr1,901054,901061,KLHL17:miR-137,14,+
